In [1]:
import time
import copy
import numpy as np
import pandas as pd
import seaborn as sn
from tqdm import tqdm
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

import sklearn
from sklearn.manifold import TSNE

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Function

from models import *
from utils import *
from datasets import *

In [2]:
class Encoder(nn.Module):

    def __init__ (self, output_dim=2):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, stride=2)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=2)
        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(128*6*6, 512)
        self.linear2 = nn.Linear(512, 512)
        self.linear3 = nn.Linear(512, output_dim)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.flatten(x)
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = F.relu(self.linear3(x))

        return x

model = Encoder()

sum(p.numel() for p in model.parameters() if p.requires_grad)

2697986

# ResNet

In [2]:
model = ResNet(BasicBlock, [1,1,1,1])
sum(p.numel() for p in model.parameters() if p.requires_grad)

85516

In [2]:
model = ResNet(BasicBlock, [1,1,1,1])
sum(p.numel() for p in model.parameters() if p.requires_grad)

321444

In [4]:
I_hat = torch.ones((1,10,100,100))

out = model(I_hat)
out.shape

torch.Size([1, 100])

# MM-II

In [4]:
model = AttnFusion(visual_size=100, metric_size=9)
visual = torch.ones((1,100))
metric = torch.ones((1,9))

feats = model(user_preference=metric, visual_feature=visual)
feats.shape

torch.Size([100])

In [7]:
answers_labels = torch.load("./results/231016000038_I_1/feedback.pt")
answers_labels, answers_labels.shape

(tensor([[0., 1., 0., 0., 0.],
         [1., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0.],
         [1., 0., 0., 0., 0.]]),
 torch.Size([4, 5]))

In [7]:
answers_pred = torch.Tensor([
    [0, 1, 0, 0, 0], 
    [1, 0, 0, 0, 0], 
    [0, 1, 0, 0, 0], 
    [1, 0, 0, 0, 0]
])

criterion = torch.nn.CrossEntropyLoss()

In [42]:
F.nll_loss(answers_labels[0], answers_pred[0].long())

tensor(0.)

In [48]:
loss = criterion(input=answers_pred[0].view(-1), target=answer_labels[0].view(-1))
loss

tensor(0.9048)

# MMModel

In [13]:
model = MMModel(MM_I_wPATH="./results/encoder_weights.pt", MM_II_wPATH=None, freeze=(False, False), device=torch.device('cpu'))

param1 = sum(p.numel() for p in model.MM_I.parameters() if p.requires_grad)
param2 = sum(p.numel() for p in model.MM_II.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters() if p.requires_grad)

param1, param2, total

(2697986, 11277259, 13975245)

In [3]:
sum(p.numel() for p in model.MM_II.cnn.parameters() if p.requires_grad)

11224164

In [4]:
resnet = ResNet(BasicBlock, [1, 1, 1, 1])
sum(p.numel() for p in resnet.parameters() if p.requires_grad)

4953444

In [5]:
img = torch.ones((1,10,1000,1000))
out = resnet(img)
out.shape

torch.Size([1, 100])

# data

In [2]:
import torchvision as tv
import torchvision.transforms as transforms
from torch.autograd import Variable
from torchvision.utils import save_image

# Data Preprocessing

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
trainTransform  = tv.transforms.Compose([tv.transforms.ToTensor(), tv.transforms.Normalize((0.1307,), (0.3081,))])
trainset = tv.datasets.MNIST(root='./data',  train=True,download=True, transform=transform)
testset = tv.datasets.MNIST(root='./data',  train=False,download=True, transform=transform)

In [3]:
traindata = [i[0] for i in trainset]
trainlabel = [i[1] for i in trainset]
testdata = [i[0] for i in testset]
testlabel = [i[1] for i in testset]

X = traindata + testdata
y = trainlabel + testlabel

In [4]:
X = torch.vstack(X)
X.shape

torch.Size([70000, 28, 28])

In [5]:
X = X.unsqueeze(1)
X.shape

torch.Size([70000, 1, 28, 28])

# MM_II

In [18]:
# z, answers = model()

z = np.load("./tsne_data_reducted_normalised.npy")
z = torch.tensor(z)
z.shape

torch.Size([70000, 3])

In [19]:
def get_Ihat(Z):
    mat = np.zeros((1000,1000))
    for i in range(len(Z)):
        xx = min(int(np.floor(Z[i,0]*1000)), 999)
        yy = min(int(np.floor(Z[i,1]*1000)), 999)
        mat[xx,yy] = Z[i, 2]
    return mat

# VI = VisualImitation(device='cpu')
# I_hat = VI(z[:128,:])
I_hat = get_Ihat(Z=z)
I_hat = torch.tensor(I_hat).unsqueeze(0)
I_hat.shape

torch.Size([1, 1000, 1000])

In [20]:
model = HumanModel(cnn_layers=[2,2,2,2], metric_num=9, hidden_dim=100)
q1, q2, q3 = model(I_hat=torch.ones((1,10,1000,1000)), z=z[:1024,:2], labels=None, x=None)

./metrics/get_scag.r


In [21]:
q1, q2, q3

(tensor([[0.5342]], grad_fn=<SigmoidBackward0>),
 tensor([[0.4964]], grad_fn=<SigmoidBackward0>),
 tensor([[0.5380]], grad_fn=<SigmoidBackward0>))

# training pipeline

In [11]:
import json

args = json.load(open("./results/231016214351_II_1/args.json"))
args

{'train': 'HM',
 'MM_I_wPATH': './results/DR_weights.pt',
 'MM_II_wPATH': None,
 'device': 'cuda',
 'batch_size_HM': 1024,
 'epochs_HM': 100,
 'feedback_path': './results/231016000038_I_1/feedback.pt',
 'exp_name': 'II_1'}

In [15]:
# model = MMModel(MM_I_wPATH="./results/encoder_weights.pt", MM_II_wPATH=None, freeze=(False, False), device=torch.device('cpu'))

model = MMModel(
    MM_I_wPATH=args["MM_I_wPATH"], 
    MM_II_wPATH=None, 
    cnn_layers=[1,1,1,1], 
    VI_size=100, 
    freeze=(True, False), 
    device=torch.device(args["device"])
)

In [17]:
# train_loader, test_loader = get_dataset(data='MNIST', DR='UMAP', args=args)




# data preprocessing
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
trainTransform  = tv.transforms.Compose([tv.transforms.ToTensor(), tv.transforms.Normalize((0.1307,), (0.3081,))])
trainset = tv.datasets.MNIST(root='./data',  train=True, download=False, transform=transform)
testset = tv.datasets.MNIST(root='./data',  train=False, download=False, transform=transform)

X_train = [i[0].unsqueeze(0) for i in trainset]
X_train = torch.vstack(X_train)
y_train = torch.tensor([i[1] for i in trainset])
print("X_train.shape: {}".format(X_train.shape))        # X_train.shape: torch.Size([60000, 1, 28, 28])

X_test = [i[0].unsqueeze(0) for i in testset]
X_test = torch.vstack(X_test)
y_test = torch.tensor([i[1] for i in testset])
print("X_test.shape: {}".format(X_test.shape))          # X_test.shape: torch.Size([10000, 1, 28, 28])




feedback = torch.load(args["feedback_path"])

train_dataset_HM = HumanDataset(X=X_train, y=y_train, feedback=feedback)
train_loader = DataLoader(train_dataset_HM, batch_size=args["batch_size_HM"], shuffle=False)

test_dataset_HM = HumanDataset(X=X_test, y=y_test, feedback=feedback)
test_loader = DataLoader(test_dataset_HM, batch_size=args["batch_size_HM"], shuffle=False)

X_train.shape: torch.Size([60000, 1, 28, 28])
X_test.shape: torch.Size([10000, 1, 28, 28])


In [18]:
for X, y, feedback in tqdm(train_loader):
    break

X.shape, y.shape, feedback.shape

  0%|          | 0/59 [00:00<?, ?it/s]


(torch.Size([1024, 1, 28, 28]), torch.Size([1024]), torch.Size([1024, 4, 5]))